In [1]:
import fitz
import os
import base64
import io
from PIL import Image

In [2]:
def get_image_caption(image_bytes: bytes) -> str:
    """VLM captioning. Replace with actual VLM call."""
    return "This is a placeholder caption for an image."

In [3]:
def table_to_markdown(tabele_data: list) -> str:
    """Converts a table (list of lists) to a Markdown format"""
    return "This is a placeholder for converted markdown of a table"

In [4]:
def process_pdf(pdf_path: str) -> str:
    """
        Processes a PDF, extracts text, images (gets captions), and tables,
        and returns a Markdown string.
    """
    doc = fitz.open(pdf_path)
    print(f"Processing PDF: {pdf_path} with {len(doc)} pages.")
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        page_content = []
        page_content.append(f"\n## Page {page_num + 1}\n")
        
        # Extracting Text
        text = page.get_text("text")
        if text.strip():
            # page_content.append("### Text\n")
            # page_content.append(text.strip())
            # page_content.append("\n")
            pass
            
        
        # Extracting Images and getting caption
        image_list = page.get_images(full=True)
        print(f"Page: {page_num}")
        # print(image_list)
        if image_list:
            print(f"YESS: {page_num}")
            page_content.append("### Images\n")
            # Write logic here to caption the images
            
        
        # Extracting tables
        # PyMuPDF's table extraction is heuristic.
        # For complex tables, check pdfplumber or camelot-py.
        tables = page.find_tables()
        if tables.tables:
            # page_content.append("### Table\n")
            # Write logic here to convert table into plain text
            pass
                    
    print(page_content)

In [31]:
PDF_PATH = 'Data/Math for AI Coding Assignment.pdf'

In [32]:
process_pdf(PDF_PATH)

Processing PDF: Data/Math for AI Coding Assignment.pdf with 16 pages.
Page: 0
Page: 1
YESS: 1
Page: 2
YESS: 2
Page: 3
YESS: 3
Page: 4
YESS: 4
Page: 5
YESS: 5
Page: 6
YESS: 6
Page: 7
YESS: 7
Page: 8
YESS: 8
Page: 9
YESS: 9
Page: 10
YESS: 10
Page: 11
YESS: 11
Page: 12
YESS: 12
Page: 13
YESS: 13
Page: 14
Page: 15
['\n## Page 16\n']


In [34]:
doc = fitz.open(PDF_PATH)
print(f"Processing PDF: {PDF_PATH} with {len(doc)} pages.")
print(doc.xref_length())

c = 0
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    
    page_content = []
    page_content.append(f"\n## Page {page_num + 1}\n")
    
    image_list = page.get_images(full=True)
    # print(f"Page: {page_num}")
    # print(image_list)
    if image_list:
        c += len(image_list)
        page_content.append("### Images\n")
        
    
print(c)

Processing PDF: Data/Math for AI Coding Assignment.pdf with 16 pages.
317
24


In [ ]:
# Geek for geeks saving images
# STEP 1
# import libraries
import fitz  # PyMuPDF
import io
from PIL import Image

# STEP 2
# file path you want to extract images from
file = "/content/pdf_file.pdf"

# open the file
pdf_file = fitz.open(file)

# STEP 3
# iterate over PDF pages
for page_index in range(len(pdf_file)):

    # get the page itself
    page = pdf_file.load_page(page_index)  # load the page
    image_list = page.get_images(full=True)  # get images on the page

    # printing number of images found in this page
    if image_list:
        print(f"[+] Found a total of {len(image_list)} images on page {page_index}")
    else:
        print("[!] No images found on page", page_index)
    
    for image_index, img in enumerate(image_list, start=1):
        # get the XREF of the image
        xref = img[0]

        # extract the image bytes
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]

        # get the image extension
        image_ext = base_image["ext"]

        # save the image
        image_name = f"image{page_index+1}_{image_index}.{image_ext}"
        with open(image_name, "wb") as image_file:
            image_file.write(image_bytes)
            print(f"[+] Image saved as {image_name}")